In [ ]:
import pandas as pd
import numpy as np
import re

df = pd.read_csv('/DataAnalyst.csv')
df.head()

,Unnamed: 0,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply
0,0,"Data Analyst, Center on Immigration and Justic...",$37K-$66K (Glassdoor est.),Are you eager to roll up your sleeves and harn...,3.2,Vera Institute of Justice\n3.2,"New York, NY","New York, NY",201 to 500 employees,1961,Nonprofit Organization,Social Assistance,Non-Profit,$100 to $500 million (USD),-1,True
1,1,Quality Data Analyst,$37K-$66K (Glassdoor est.),Overview\n\nProvides analytical and technical ...,3.8,Visiting Nurse Service of New York\n3.8,"New York, NY","New York, NY",10000+ employees,1893,Nonprofit Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),-1,-1
2,2,"Senior Data Analyst, Insights & Analytics Team...",$37K-$66K (Glassdoor est.),We’re looking for a Senior Data Analyst who ha...,3.4,Squarespace\n3.4,"New York, NY","New York, NY",1001 to 5000 employees,2003,Company - Private,Internet,Information Technology,Unknown / Non-Applicable,GoDaddy,-1
3,3,Data Analyst,$37K-$66K (Glassdoor est.),Requisition NumberRR-0001939\nRemote:Yes\nWe c...,4.1,Celerity\n4.1,"New York, NY","McLean, VA",201 to 500 employees,2002,Subsidiary or Business Segment,IT Services,Information Technology,$50 to $100 million (USD),-1,-1
4,4,Reporting Data Analyst,$37K-$66K (Glassdoor est.),ABOUT FANDUEL GROUP\n\nFanDuel Group is a worl...,3.9,FanDuel\n3.9,"New York, NY","New York, NY",501 to 1000 employees,2009,Company - Private,Sports & Recreation,"Arts, Entertainment & Recreation",$100 to $500 million (USD),DraftKings,True


In [ ]:
print(f'Dataset shape: {df.shape}')

print("\nColumn names:")
for i, col in enumerate(df.columns):
  print(f"{i}: {col}")

print("\nMissing values:")
print(df.isnull().sum())

Dataset shape: (2253, 16)

Column names:
0: Unnamed: 0
1: Job Title
2: Salary Estimate
3: Job Description
4: Rating
5: Company Name
6: Location
7: Headquarters
8: Size
9: Founded
10: Type of ownership
11: Industry
12: Sector
13: Revenue
14: Competitors
15: Easy Apply

Missing values:
Unnamed: 0           0
Job Title            0
Salary Estimate      0
Job Description      0
Rating               0
Company Name         1
Location             0
Headquarters         0
Size                 0
Founded              0
Type of ownership    0
Industry             0
Sector               0
Revenue              0
Competitors          0
Easy Apply           0
dtype: int64


In [ ]:
columns_to_drop = ['Unnamed: 0', 'Competitors', 'Easy Apply']

df = df.drop(columns=columns_to_drop)

df = df.dropna()

# Look at salary column
print(f"New shape: {df.shape}")
print(f"Columns: {len(df.columns)}")

# Check for hourly rates
hourly_counts = df['Salary Estimate'].str.contains('Per Hour', na=False).sum()
print(f'\nJobs with hourly rates: {hourly_counts}')

New shape: (2252, 13)
Columns: 13

Jobs with hourly rates: 0


In [ ]:
df['Salary Estimate'].value_counts()

,count
Salary Estimate,
$41K-$78K (Glassdoor est.),57
$42K-$76K (Glassdoor est.),57
$50K-$86K (Glassdoor est.),41
$35K-$67K (Glassdoor est.),33
$60K-$124K (Glassdoor est.),31
...,...
$47K-$81K (Glassdoor est.),3
$36K-$67K (Glassdoor est.),3
$43K-$77K (Glassdoor est.),3


In [ ]:
# Extract salary numbers
df[['Salary_Min', 'Salary_Max']] = df['Salary Estimate'].str.extract(r'\$(\d+)K-\$(\d+)K').astype(float)

# Find average
df['Salary_Avg'] = (df['Salary_Min'] + df['Salary_Max']) / 2

# Drop original column
df = df.drop('Salary Estimate', axis=1)

print(f'Salary Range: ${df['Salary_Min'].min()}-${df['Salary_Max'].max()}K')

Salary Range: $24.0-$190.0K


In [ ]:
skills = {
    'SQL': ['sql', 'mysql', 'postgresql', 't-sql', 'tsql', 'pl/sql'],
    'Python': ['python', 'pandas', 'numpy', 'scikit'],
    'R': [' r ', ' r,', ' r.', 'rstudio', 'ggplot', ' r)'],
    'Excel': ['excel', 'spreadsheet', 'vba'],
    'Tableau': ['tableau'],
    'Power_BI': ['power bi', 'powerbi', 'power-bi'],
    'SAS': ['sas '],
    'Machine_Learning': ['machine learning', 'ml', 'deep learning', 'neural network', 'scikit-learn'],
    'Statistics': ['statistics', 'statistical', 'regression', 'hypothesis'],
    'AWS': ['aws', 'amazon web services', 's3', 'ec2'],
    'Spark': ['spark', 'pyspark'],
    'Java': ['java '],
    'Azure': ['azure', 'microsoft cloud']
}

print("Skills we're searching for:")
for skill in skills.keys():
    print(f"  - {skill}")

Skills we're searching for:
  - SQL
  - Python
  - R
  - Excel
  - Tableau
  - Power_BI
  - SAS
  - Machine_Learning
  - Statistics
  - AWS
  - Spark
  - Java
  - Azure


In [ ]:
# Function to check if a skill exists in job description
def check_skill(text, skill_keywords):
    """
    Checks if any keyword from the skill list appears in the text
    Returns 1 if found, 0 if not
    """
    if pd.isna(text):  # Handle missing descriptions
        return 0

    text_lower = text.lower()  # Convert to lowercase for matching

    for keyword in skill_keywords:
        if keyword in text_lower:
            return 1

    return 0

# Test it on one job description
sample_desc = df['Job Description'].iloc[0]
print("Testing on first job description:")
print(f"\nHas SQL? {check_skill(sample_desc, skills['SQL'])}")
print(f"Has Python? {check_skill(sample_desc, skills['Python'])}")
print(f"Has Tableau? {check_skill(sample_desc, skills['Tableau'])}")

Testing on first job description:

Has SQL? 1
Has Python? 1
Has Tableau? 0


In [ ]:
for skill_name, keywords in skills.items():
    df[skill_name] = df['Job Description'].apply(lambda x: check_skill(x, keywords))

for skill_name in skills.keys():
  count = df[skill_name].sum()
  percentage = (count / len(df)) * 100
  print(f"{skill_name:20} {count:4} jobs ({percentage:5.1f}%)")

print(f"\n{'='*50}")
print(f"Total jobs analyzed: {len(df)}")

SQL                  1388 jobs ( 61.6%)
Python                638 jobs ( 28.3%)
R                     337 jobs ( 15.0%)
Excel                1372 jobs ( 60.9%)
Tableau               620 jobs ( 27.5%)
Power_BI              249 jobs ( 11.1%)
SAS                   159 jobs (  7.1%)
Machine_Learning      469 jobs ( 20.8%)
Statistics            856 jobs ( 38.0%)
AWS                   284 jobs ( 12.6%)
Spark                  89 jobs (  4.0%)
Java                   32 jobs (  1.4%)
Azure                  67 jobs (  3.0%)

Total jobs analyzed: 2252


In [ ]:
# Split location into city and state
df[['City', 'State']] = df['Location'].str.split(',', n=1, expand=True)

# Clean up state (remove whitespace)
df['State'] = df['State'].str.strip()

# Check results
print("Top 10 states with most jobs:")
print(df['State'].value_counts().head(10))
print(f"\nTop 10 cities:")
print(df['City'].value_counts().head(10))

Top 10 states with most jobs:
State
CA    626
TX    394
NY    345
IL    164
PA    114
AZ     97
NC     89
CO     88
NJ     86
WA     54
Name: count, dtype: int64

Top 10 cities:
City
New York         310
Chicago          130
San Francisco    119
Austin            81
Los Angeles       80
Charlotte         78
Houston           72
Dallas            66
San Diego         62
Philadelphia      56
Name: count, dtype: int64


In [ ]:
# Categorize by seniority level
def categorize_seniority(title):
    """Categorizes job by seniority based on title keywords"""
    if pd.isna(title):
        return 'Unknown'

    title_lower = title.lower()

    # Senior indicators
    if any(word in title_lower for word in ['senior', 'sr.', 'sr ', 'lead', 'principal', 'staff', 'manager']):
        return 'Senior'

    # Entry level indicators
    elif any(word in title_lower for word in ['junior', 'jr.', 'jr ', 'entry', 'associate', 'intern']):
        return 'Entry Level'

    # Everything else is mid-level
    else:
        return 'Mid Level'

# Apply to all jobs
df['Seniority'] = df['Job Title'].apply(categorize_seniority)

# Check distribution
print("Job seniority breakdown:")
print(df['Seniority'].value_counts())
print(f"\nPercentages:")
print(df['Seniority'].value_counts(normalize=True) * 100)

Job seniority breakdown:
Seniority
Mid Level      1638
Senior          504
Entry Level     110
Name: count, dtype: int64

Percentages:
Seniority
Mid Level      72.735346
Senior         22.380107
Entry Level     4.884547
Name: proportion, dtype: float64


In [ ]:
# Select columns for Tableau
columns_for_tableau = [
    'Job Title',
    'Salary_Min',
    'Salary_Max',
    'Salary_Avg',
    'Company Name',
    'Location',
    'City',
    'State',
    'Rating',
    'Size',
    'Sector',
    'Industry',
    'Seniority',
    # All skill columns
    'SQL', 'Python', 'R', 'Excel', 'Tableau', 'Power_BI',
    'SAS', 'Machine_Learning', 'Statistics', 'AWS', 'Spark', 'Java', 'Azure'
]

df_clean = df[columns_for_tableau]

# Export
df_clean.to_csv('DataAnalyst_Cleaned.csv', index=False)

from google.colab import files
files.download('DataAnalyst_Cleaned.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>